In [1]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
import sys
import os
import tsfel
from joblib import Parallel, delayed
import multiprocessing
import time


In [2]:
#minimum 6 months of data needed

In [3]:
!pip show tsfel

Name: tsfel
Version: 0.1.9
Summary: Library for time series feature extraction
Home-page: https://github.com/fraunhoferportugal/tsfel/
Author: Fraunhofer Portugal
Author-email: 
License: 
Location: c:\users\vaseekaranv\appdata\local\anaconda3\envs\nibm_dl\lib\site-packages
Requires: ipython, numpy, pandas, PyWavelets, requests, scikit-learn, scipy, setuptools, statsmodels
Required-by: 


In [4]:
# Setting path to load util functions
from pathlib import Path
parent_dir = Path.cwd().parents[1]
sys.path.append(os.path.abspath(parent_dir))

In [5]:
selected_features = pd.read_csv('../../utils/selected_features/boruta_features_baseline.csv')
selected_features = selected_features[selected_features['Importance'] == 1]['Feature'].values

In [6]:
cols_to_append = ['customer_ID', 'end_of_month']
cols_to_append = np.append(cols_to_append, selected_features)
cols_to_append

array(['customer_ID', 'end_of_month', 'D_63_CL', 'B_14', 'D_60', 'S_11',
       'D_65', 'B_18', 'R_6', 'S_13', 'D_71', 'S_15', 'B_23', 'D_58',
       'P_4', 'R_10', 'R_11', 'R_12', 'B_28', 'R_16', 'S_26', 'D_102',
       'B_37', 'B_40', 'D_133', 'D_75', 'R_5', 'B_24', 'B_12', 'D_63_CO',
       'D_63_CR', 'S_8', 'D_126_1.0', 'D_39', 'B_1', 'R_1', 'B_4', 'B_5',
       'R_2', 'D_47', 'D_144', 'D_51', 'B_11', 'S_5', 'R_3', 'B_9',
       'B_10', 'R_4', 'B_7'], dtype=object)

In [7]:
df = pd.read_parquet('../../data/non-null/df_13_months_unprocessed_non_null.parquet').drop(columns=['S_2', 'target'])

In [8]:
train_ids = pd.read_parquet('../../data/3d_array/train_targets.parquet')['customer_ID'].values
val_ids = pd.read_parquet('../../data/3d_array/val_targets.parquet')['customer_ID'].values
test_ids = pd.read_parquet('../../data/3d_array/test_targets.parquet')['customer_ID'].values

In [9]:
train_df = df[df['customer_ID'].isin(train_ids)]
val_df = df[df['customer_ID'].isin(val_ids)]
test_df = df[df['customer_ID'].isin(test_ids)]

In [10]:
df = None

In [11]:
train_df.head()

,customer_ID,D_39,B_1,R_1,B_4,B_5,R_2,D_47,B_7,D_51,B_9,R_3,B_10,S_5,B_11,S_6,R_4,B_12,S_8,R_5,D_58,B_14,D_60,S_11,D_63,D_65,B_18,S_12,R_6,S_13,B_21,D_71,S_15,B_23,P_4,D_75,B_24,R_7,R_8,S_16,R_10,R_11,S_17,R_12,B_28,R_13,R_14,R_15,R_16,S_18,D_86,R_17,R_18,B_31,S_19,R_19,B_32,S_20,R_20,R_21,R_22,R_23,D_92,D_93,D_94,R_24,R_25,D_96,S_26,D_102,B_36,B_37,B_40,D_126,D_127,B_41,D_133,R_28,D_140,D_144,end_of_month
0,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,0.001733,0.008724,0.009228,0.080986,0.170600,0.006204,0.525351,0.059416,1.335856,0.008207,0.001423,0.096219,0.023381,0.002768,0.008322,0.008298,0.148266,0.922998,0.001882,0.158612,0.018385,0.199617,0.401619,CR,0.007126,0.652984,0.272008,0.008363,0.515222,0.002644,0.119403,0.108271,0.050882,0.007554,0.069067,0.004327,0.007562,0.001434,0.002271,0.007121,0.002456,0.008033,1.009825,0.084683,0.003820,0.000438,0.006452,0.005055,0.005720,0.007084,0.000198,0.008907,1,0.002537,0.005177,0.006626,0.009705,0.007782,0.002450,0.007479,0.006893,1.006133,0.003569,0.008871,0.003950,0.003647,0.004950,0.001243,0.766688,0.009968,0.004572,0.210060,1.0,1.003319,0.006805,0.004345,0.001535,0.003706,0.000610,2017-03-31
1,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,0.005775,0.004923,0.006151,0.069419,0.113239,0.006206,0.521311,0.057744,1.339794,0.008373,0.001984,0.099804,0.030599,0.002749,0.002482,0.005136,0.143530,0.919414,0.001610,0.148459,0.013035,0.151387,0.406326,CR,0.002413,0.647093,0.188970,0.004030,0.509048,0.004193,0.140611,0.101018,0.040469,0.004832,0.074166,0.004203,0.005304,0.000509,0.009810,0.005966,0.000395,0.000760,1.009461,0.081843,0.000347,0.004311,0.002332,0.003753,0.007584,0.006677,0.001142,0.005907,1,0.008427,0.008979,0.001854,0.009924,0.005987,0.002247,0.006827,0.002837,1.005791,0.000571,0.000391,0.008351,0.008850,0.003180,0.004561,0.786007,0.003921,0.004654,0.184093,1.0,1.008394,0.004407,0.007495,0.004931,0.003167,0.005492,2017-04-30
2,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,0.091505,0.021655,0.006815,0.068839,0.060492,0.003259,0.524568,0.056647,1.337179,0.009355,0.007426,0.134073,0.048367,0.010077,0.000530,0.006961,0.137014,1.001977,0.006328,0.139504,0.056653,0.305883,0.406768,CR,0.001878,0.645819,0.495308,0.006838,0.679257,0.001337,0.075868,0.103239,0.047454,0.006561,0.076510,0.001782,0.001422,0.008295,0.009362,0.005447,0.007345,0.004056,1.004291,0.081954,0.002709,0.007139,0.008358,0.007381,0.005901,0.001185,0.008013,0.008882,1,0.007327,0.002016,0.008686,0.008446,0.007291,0.007794,0.009820,0.005080,1.005801,0.007425,0.009234,0.002471,0.009769,0.005433,0.011736,0.806840,0.001264,0.019176,0.154837,1.0,1.009307,0.003221,0.009227,0.009123,0.007329,0.006986,2017-05-31
3,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,0.002455,0.013683,0.001373,0.055630,0.166782,0.009918,0.530929,0.049253,1.339909,0.006782,0.003515,0.134437,0.030063,0.009667,0.000783,0.008706,0.129017,0.704016,0.004980,0.138100,0.012498,0.273553,0.405175,CR,0.005899,0.654358,0.508670,0.008183,0.515282,0.008716,0.150209,0.206394,0.031705,0.009559,0.071547,0.005595,0.006363,0.005153,0.004876,0.001888,0.004961,0.006969,1.004728,0.060634,0.009982,0.008690,0.007364,0.008802,0.002520,0.003324,0.009455,0.008348,1,0.007053,0.003909,0.002478,0.006614,0.009977,0.007686,0.000458,0.007320,1.007036,0.000664,0.003200,0.008507,0.004858,0.000063,0.007571,0.808214,0.002729,0.011720,0.153939,1.0,1.001671,0.007703,0.007206,0.002409,0.004516,0.006527,2017-06-30
4,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,0.002483,0.015193,0.007605,0.038862,0.143630,0.006667,0.529305,0.048918,1.341735,0.000519,0.001362,0.121518,0.054221,0.009484,0.006698,0.003846,0.129539,0.917133,0.001653,0.126443,0.027897,0.233103,0.487460,CR,0.009479,0.650112,0.216507,0.008605,0.507712,0.006821,0.096441,0.106020,0.032733,0.008156,0.074432,0.004933,0.004831,0.007338,0.007447,0.006111,0.002246,0.001770,1.000904,0.062492,0.005860,0.007816,0.002470,0.007166,0.000155,0.001504,0.002019,0.002678,1,0.007728,0.003432,0.002199,0.005511,0.004105,0.009

In [12]:
train_df.shape

(3486098, 81)

In [13]:
from sklearn.preprocessing import OneHotEncoder
import pandas as pd
import numpy as np

def one_hot_encode_categories(df, categorical_columns=None, drop_original=True, handle_unknown='error'):
    """
    One-hot encode categorical columns in a DataFrame.
    
    Parameters:
    -----------
    df : pandas.DataFrame
        Input DataFrame containing categorical columns
    categorical_columns : list or None
        List of categorical column names to encode. If None, automatically detects categorical columns.
    drop_original : bool
        Whether to drop the original categorical columns
    handle_unknown : str
        Strategy for handling unknown categories in new data: 'error', 'ignore' or 'infrequent_if_exist'
    
    Returns:
    --------
    pandas.DataFrame
        DataFrame with one-hot encoded columns
    OneHotEncoder
        Fitted encoder for future transformations
    """
    # Make a copy to avoid modifying the original
    result_df = df.copy()
    
    # Automatically detect categorical columns if not specified
    if categorical_columns is None:
        categorical_columns = result_df.select_dtypes(include=['object', 'category']).columns.tolist()
    
    if not categorical_columns:
        print("No categorical columns found to encode.")
        return result_df, None
        
    # Initialize the encoder
    encoder = OneHotEncoder(sparse_output=False, handle_unknown=handle_unknown, drop='if_binary')
    
    # Fit and transform the categorical columns
    encoded_array = encoder.fit_transform(result_df[categorical_columns])
    
    # Get feature names
    feature_names = encoder.get_feature_names_out(categorical_columns)
    
    # Create a DataFrame with the encoded features
    encoded_df = pd.DataFrame(encoded_array, columns=feature_names, index=result_df.index)
    
    # Combine with the original DataFrame
    if drop_original:
        # Drop the original categorical columns
        result_df = result_df.drop(columns=categorical_columns)
    
    # Concatenate the encoded columns with the original DataFrame
    result_df = pd.concat([result_df, encoded_df], axis=1)
    
    print(f"One-hot encoded {len(categorical_columns)} categorical columns into {len(feature_names)} binary features.")
    
    return result_df, encoder

In [14]:
cat_cols = ['D_126', 'D_63']
train_df[cat_cols] = train_df[cat_cols].astype('category')
val_df[cat_cols] = val_df[cat_cols].astype('category')
test_df[cat_cols] = test_df[cat_cols].astype('category')

In [15]:
ohe = OneHotEncoder(handle_unknown='ignore', sparse_output=False, drop='if_binary')
ohe.fit_transform(train_df[cat_cols])

array([[0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       ...,
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.]])

In [16]:
train_df, cat_encoder = one_hot_encode_categories(train_df, handle_unknown='ignore', categorical_columns=cat_cols)

One-hot encoded 2 categorical columns into 10 binary features.


In [17]:
val_df = pd.concat([val_df.drop(columns=cat_encoder.feature_names_in_),pd.DataFrame(cat_encoder.transform(val_df[cat_encoder.feature_names_in_]), columns=cat_encoder.get_feature_names_out(),index=val_df.index)], axis=1)
test_df = pd.concat([test_df.drop(columns=cat_encoder.feature_names_in_),pd.DataFrame(cat_encoder.transform(test_df[cat_encoder.feature_names_in_]), columns=cat_encoder.get_feature_names_out(),index=test_df.index)], axis=1)

In [18]:
from multiprocessing import Pool
import time
import gc # Import garbage collector

In [19]:
def process_customer_group_for_joblib(group_info, cfg, id_col, ts_col):
    """
    Applies TSFEL feature extraction to a single customer's data.
    Designed to be called by joblib.Parallel.
    Includes verbose prints for skipped customers.

    Args:
        group_info (tuple): A tuple containing (customer_id, customer_data_df).
        cfg (dict): The TSFEL configuration dictionary.
        id_col (str): Name of the customer ID column.
        ts_col (str): Name of the timestamp column.

    Returns:
        pd.DataFrame or None: A DataFrame containing the extracted features for the customer
                              (indexed by customer ID), or None if processing fails/skipped.
    """
    customer_id, customer_data = group_info # Unpack the tuple from groupby

    try:
        # --- Data Validation / Column Selection ---
        if len(customer_data) < 6:
            # print(f"Skipping {customer_id}: Insufficient data points ({len(customer_data)}).") # Verbose skipping
            return None

        numeric_cols = customer_data.select_dtypes(include=['number']).columns
        cols_to_process = [col for col in numeric_cols if col not in [id_col, ts_col]]

        if not cols_to_process:
             # print(f"Skipping {customer_id}: No valid numeric columns found.") # Verbose skipping
            return None

        # --- TSFEL Extraction ---
        features_df = tsfel.time_series_features_extractor(cfg, customer_data[cols_to_process], verbose=0)

        # --- Result Handling ---
        if features_df.empty:
            # print(f"Skipping {customer_id}: TSFEL returned empty features.") # Verbose skipping
            return None

        # Set index *before* returning
        features_df.index = [customer_id] * len(features_df)
        features_df.index.name = id_col

        return features_df

    except Exception as e:
        print(f"Error processing customer {customer_id}: {type(e).__name__} - {e}")
        return None

In [20]:
def extract_tsfel_features_joblib_optimized(df, customer_id_col, timestamp_col, features_to_extract=None, n_jobs=-1, verbose=10):
    """
    Extracts time-series features using TSFEL in parallel using joblib
    (provides progress updates & ETA), with robust concatenation using reindexing.

    Args:
        df (pd.DataFrame): Input DataFrame.
        customer_id_col (str): Customer ID column name.
        timestamp_col (str): Timestamp column name.
        features_to_extract (dict, optional): TSFEL config dict. Defaults to all features.
        n_jobs (int, optional): Number of parallel processes (-1 for all cores). Defaults to -1.
        verbose (int, optional): Joblib verbosity level. Higher values (e.g., 5, 10)
                                 show progress bars and ETA. Defaults to 10.

    Returns:
        pd.DataFrame: DataFrame with customer_ID as index and TSFEL features as columns.
                      Returns an empty DataFrame on major errors or if no features are extracted.
    """
    if df.empty:
        print("Input DataFrame is empty.")
        return pd.DataFrame()

    print(f"Input shape: {df.shape}, Grouping by: {customer_id_col}, Timestamp: {timestamp_col}")

    # --- Data Preparation ---
    df_sorted = df.sort_values(by=[customer_id_col, timestamp_col])

    # --- TSFEL Configuration ---
    # (Configuration logic remains the same)
    if features_to_extract is None:
        cfg = tsfel.get_features_by_domain()
        print("Using default (all) TSFEL feature configuration.")
    elif isinstance(features_to_extract, dict):
        cfg = features_to_extract
        print("Using provided TSFEL configuration dictionary.")
    else:
         print("Warning: Invalid 'features_to_extract' format. Using default config.")
         cfg = tsfel.get_features_by_domain()

    # --- Grouping ---
    grouped_data = df_sorted.groupby(customer_id_col)
    num_groups = len(grouped_data)
    if num_groups == 0:
        print("No groups found for the specified customer ID.")
        del df_sorted
        gc.collect()
        return pd.DataFrame()
    print(f"Processing {num_groups} customer groups...")

    del df_sorted
    gc.collect()

    # --- Determine Number of Processes ---
    if n_jobs == -1:
        num_cores = multiprocessing.cpu_count()
    else:
        num_cores = min(n_jobs, multiprocessing.cpu_count())
    print(f"Setting up Joblib with {num_cores} processes (verbose={verbose})...")


    # --- Parallel Execution using joblib.Parallel ---
    start_time = time.time()
    results = Parallel(n_jobs=num_cores, verbose=verbose)(
        delayed(process_customer_group_for_joblib)(
            group_info,
            cfg,
            customer_id_col,
            timestamp_col
        )
        for group_info in grouped_data
    )
    end_time = time.time()
    print(f"\nJoblib parallel processing finished in {end_time - start_time:.2f} seconds.")


    # --- Combine Results (Robust Method using Reindexing) ---
    start_concat_time = time.time()
    successful_results = [res for res in results if res is not None and not res.empty]
    del results
    gc.collect()

    if not successful_results:
        print("No features were successfully extracted for any customer.")
        return pd.DataFrame()

    print(f"Successfully processed {len(successful_results)} out of {num_groups} groups.")
    print("Preparing results for concatenation...")

    try:
        # 1. Find the union of all columns from all successful results
        all_columns = set()
        for df_res in successful_results:
            all_columns.update(df_res.columns)
        all_columns = sorted(list(all_columns)) # Sort for consistent order

        # 2. Reindex each result DataFrame to include all columns, filling missing with NaN
        #    This ensures all DataFrames have the same structure before concatenation.
        reindexed_results = [
            df_res.reindex(columns=all_columns, fill_value=np.nan)
            for df_res in successful_results
        ]
        del successful_results # Free memory from original list
        gc.collect()

        # 3. Concatenate the reindexed DataFrames. pd.concat is efficient here.
        print("Concatenating reindexed results...")
        final_features_df = pd.concat(reindexed_results)

        # Index name should be preserved from worker function, but set again just in case
        final_features_df.index.name = customer_id_col

        del reindexed_results # Free memory from reindexed list
        gc.collect()

        print("Concatenation complete.")
        end_concat_time = time.time()
        print(f"Robust concatenation finished in {end_concat_time - start_concat_time:.2f} seconds.")

    except Exception as e:
        print(f"Error during robust concatenation: {e}")
        # Clean up potential large lists
        try: del successful_results
        except NameError: pass
        try: del reindexed_results
        except NameError: pass
        gc.collect()
        return pd.DataFrame()

    print(f"Final features DataFrame shape: {final_features_df.shape}")
    return final_features_df

In [21]:
cfg_stat = tsfel.get_features_by_domain(domain=['statistical'])

In [23]:
features_df_joblib_opt = extract_tsfel_features_joblib_optimized(
        val_df[cols_to_append],
        customer_id_col='customer_ID',
        timestamp_col='end_of_month',
        features_to_extract=cfg_stat,
        n_jobs=-1, # Use all cores
        verbose=10 # Set verbosity for progress updates and ETA
    )

# --- Display Results ---
print("\n--- Extracted Features (Optimized Joblib Method + Verbose Progress) ---")
if not features_df_joblib_opt.empty:
    print(f"Shape: {features_df_joblib_opt.shape}")
    try:
        display(features_df_joblib_opt.head())
    except NameError:
        print(features_df_joblib_opt.head())
else:
    print("Resulting DataFrame is empty.")

Input shape: (387137, 49), Grouping by: customer_ID, Timestamp: end_of_month
Using provided TSFEL configuration dictionary.
Processing 32124 customer groups...
Setting up Joblib with 20 processes (verbose=10)...


[Parallel(n_jobs=20)]: Using backend LokyBackend with 20 concurrent workers.
[Parallel(n_jobs=20)]: Done   1 tasks      | elapsed:    9.3s
[Parallel(n_jobs=20)]: Done  10 tasks      | elapsed:    9.4s
[Parallel(n_jobs=20)]: Done  21 tasks      | elapsed:    9.5s
[Parallel(n_jobs=20)]: Done  32 tasks      | elapsed:    9.7s
[Parallel(n_jobs=20)]: Done  45 tasks      | elapsed:    9.8s
[Parallel(n_jobs=20)]: Done  58 tasks      | elapsed:    9.9s
[Parallel(n_jobs=20)]: Done  73 tasks      | elapsed:   10.0s
[Parallel(n_jobs=20)]: Done  88 tasks      | elapsed:   10.0s
[Parallel(n_jobs=20)]: Done 105 tasks      | elapsed:   10.1s
[Parallel(n_jobs=20)]: Done 122 tasks      | elapsed:   10.2s
[Parallel(n_jobs=20)]: Done 141 tasks      | elapsed:   10.3s
[Parallel(n_jobs=20)]: Batch computation too fast (0.19554920884668023s.) Setting batch_size=2.
[Parallel(n_jobs=20)]: Done 160 tasks      | elapsed:   10.4s
[Parallel(n_jobs=20)]: Done 183 tasks      | elapsed:   10.6s
[Parallel(n_jobs=20)]


Joblib parallel processing finished in 162.30 seconds.
Successfully processed 28164 out of 32124 groups.
Preparing results for concatenation...
Concatenating reindexed results...
Concatenation complete.
Robust concatenation finished in 97.94 seconds.
Final features DataFrame shape: (28164, 1457)

--- Extracted Features (Optimized Joblib Method + Verbose Progress) ---
Shape: (28164, 1457)


B_10_Absolute energy  \
customer_ID                                                                
0006a80d96aec1d178bf74adab6cbf9e4bf427a6ba264fe...              0.061933   
000f8675ede66cc6affd4c048db11a00246d7ee623f453a...              0.755592   
0014eef7b7aefc114ea48847275fe8e5c14455e5adb43a7...              0.021994   
0016b9485e459e660a7b05fa1df47c5e206bb3e62836b9c...              0.494161   
0019b0b705b2aa3aaaec7e1d74b829dee8a38502907597d...              0.013737   

                                                    B_10_Average power  \
customer_ID                                                              
0006a80d96aec1d178bf74adab6cbf9e4bf427a6ba264fe...            0.516107   
000f8675ede66cc6affd4c048db11a00246d7ee623f453a...            6.296602   
0014eef7b7aefc114ea48847275fe8e5c14455e5adb43a7...            0.183287   
0016b9485e459e660a7b05fa1df47c5e206bb3e62836b9c...            4.118011   
0019b0b705b2aa3aaaec7e1d74b829dee8a38502907597d...            0.114476   

                                                    B_10_ECDF Percentile Count_0  \
customer_ID                                                                        
0006a80d96aec1d178bf74adab6cbf9e4bf427a6ba264fe...                           2.0   
000f8675ede66cc6affd4c048db11a00246d7ee623f453a...                           2.0   
0014eef7b7aefc114ea48847275fe8e5c14455e5adb43a7...                           2.0   
0016b9485e459e660a7b05fa1df47c5e206bb3e62836b9c...                           2.0   
0019b0b705b2aa3aaaec7e1d74b829dee8a38502907597d...                           2.0   

                                                    B_10_ECDF Percentile Count_1  \
customer_ID                                                                        
0006a80d96aec1d178bf74adab6cbf9e4bf427a6ba264fe...                          10.0   
000f8675ede66cc6affd4c048db11a00246d7ee623f453a...                          10.0   
0014eef7b7aefc114ea48847275fe8e5c14455e5adb43a7...                          10.0   
0016b9485e459e660a7b05fa1df47c5e206bb3e62836b9c...                          10.0   
0019b0b705b2aa3aaaec7e1d74b829dee8a38502907597d...                          10.0   

                                                    B_10_ECDF Percentile_0  \
customer_ID                                                                  
0006a80d96aec1d178bf74adab6cbf9e4bf427a6ba264fe...                0.022246   
000f8675ede66cc6affd4c048db11a00246d7ee623f453a...                0.238390   
0014eef7b7aefc114ea48847275fe8e5c14455e5adb43a7...                0.029578   
0016b9485e459e660a7b05fa1df47c5e206bb3e62836b9c...                0.050188   
0019b0b705b2aa3aaaec7e1d74b829dee8a38502907597d...                0.024693   

                                                    B_10_ECDF Percentile_1  \
customer_ID                                                                  
0006a80d96aec1d178bf74adab6cbf9e4bf427a6ba264fe...                0.070988   
000f8675ede66cc6affd4c048db11a00246d7ee623f453a...                0.242116   
0014eef7b7aefc114ea48847275fe8e5c14455e5adb43a7...                0.047638   
0016b9485e459e660a7b05fa1df47c5e206bb3e62836b9c...                0.297941   
0019b0b705b2aa3aaaec7e1d74b829dee8a38502907597d...                0.035624   

                                                    B_10_ECDF_0  B_10_ECDF_1  \
customer_ID                                                                    
0006a80d96aec1d178bf74adab6cbf9e4bf427a6ba264fe...     0.076923     0.153846   
000f8675ede66cc6affd4c048db11a00246d7ee623f453a...     0.076923     0.153846   
0014eef7b7aefc114ea48847275fe8e5c14455e5adb43a7...     0.076923     0.153846   
0016b9485e459e660a7b05fa1df47c5e206bb3e62836b9c...     0.076923     0.153846   
0019b0b705b2aa3aaaec7e1d74b829dee8a38502907597d...     0.076923     0.153846   

                                                    B_10_ECDF_2  B_10_ECDF_3  \
customer_ID                                                                    
0006a80

In [24]:
features_df_joblib_opt.to_parquet('../../data/tsfel_features/val_13_months_boruta_selected_tsfel_stat_features.parquet', index=True, engine='pyarrow', compression='zstd')

In [25]:
features_df_joblib_opt = extract_tsfel_features_joblib_optimized(
        test_df[cols_to_append],
        customer_id_col='customer_ID',
        timestamp_col='end_of_month',
        features_to_extract=cfg_stat,
        n_jobs=-1, # Use all cores
        verbose=10 # Set verbosity for progress updates and ETA
    )

# --- Display Results ---
print("\n--- Extracted Features (Optimized Joblib Method + Verbose Progress) ---")
if not features_df_joblib_opt.empty:
    print(f"Shape: {features_df_joblib_opt.shape}")
    try:
        display(features_df_joblib_opt.head())
    except NameError:
        print(features_df_joblib_opt.head())
else:
    print("Resulting DataFrame is empty.")

Input shape: (1658216, 49), Grouping by: customer_ID, Timestamp: end_of_month
Using provided TSFEL configuration dictionary.
Processing 137674 customer groups...
Setting up Joblib with 20 processes (verbose=10)...


[Parallel(n_jobs=20)]: Using backend LokyBackend with 20 concurrent workers.
[Parallel(n_jobs=20)]: Done   1 tasks      | elapsed:    0.1s
[Parallel(n_jobs=20)]: Batch computation too fast (0.011857271194458008s.) Setting batch_size=2.
[Parallel(n_jobs=20)]: Done  10 tasks      | elapsed:    0.2s
[Parallel(n_jobs=20)]: Done  21 tasks      | elapsed:    0.2s
[Parallel(n_jobs=20)]: Done  32 tasks      | elapsed:    0.3s
[Parallel(n_jobs=20)]: Batch computation too fast (0.1898818016052246s.) Setting batch_size=4.
[Parallel(n_jobs=20)]: Done  50 tasks      | elapsed:    0.4s
[Parallel(n_jobs=20)]: Done  76 tasks      | elapsed:    0.5s
[Parallel(n_jobs=20)]: Done 106 tasks      | elapsed:    0.6s
[Parallel(n_jobs=20)]: Done 152 tasks      | elapsed:    0.9s
[Parallel(n_jobs=20)]: Done 220 tasks      | elapsed:    1.2s
[Parallel(n_jobs=20)]: Done 288 tasks      | elapsed:    1.4s
[Parallel(n_jobs=20)]: Done 364 tasks      | elapsed:    1.8s
[Parallel(n_jobs=20)]: Done 440 tasks      | elap


Joblib parallel processing finished in 653.32 seconds.
Successfully processed 120574 out of 137674 groups.
Preparing results for concatenation...
Concatenating reindexed results...
Concatenation complete.
Robust concatenation finished in 601.42 seconds.
Final features DataFrame shape: (120574, 1457)

--- Extracted Features (Optimized Joblib Method + Verbose Progress) ---
Shape: (120574, 1457)


B_10_Absolute energy  \
customer_ID                                                                
00000fd6641609c6ece5454664794f0340ad84dddce9a26...              1.160887   
000041bdba6ecadd89a52d11886e8eaaec9325906c97233...              1.297565   
00007889e4fcd2614b6cbe7f8f3d2e5c728eca32d9eb8ad...              0.197565   
0000d17a1447b25a01e42e1ac56b091bb7cbb06317be4cb...              0.838148   
00013181a0c5fc8f1ea38cd2b90fe8ad2fa8cad9d9f13e4...              0.007884   

                                                    B_10_Average power  \
customer_ID                                                              
00000fd6641609c6ece5454664794f0340ad84dddce9a26...            9.674057   
000041bdba6ecadd89a52d11886e8eaaec9325906c97233...           10.813043   
00007889e4fcd2614b6cbe7f8f3d2e5c728eca32d9eb8ad...            1.646377   
0000d17a1447b25a01e42e1ac56b091bb7cbb06317be4cb...            6.984564   
00013181a0c5fc8f1ea38cd2b90fe8ad2fa8cad9d9f13e4...            0.065703   

                                                    B_10_ECDF Percentile Count_0  \
customer_ID                                                                        
00000fd6641609c6ece5454664794f0340ad84dddce9a26...                           2.0   
000041bdba6ecadd89a52d11886e8eaaec9325906c97233...                           2.0   
00007889e4fcd2614b6cbe7f8f3d2e5c728eca32d9eb8ad...                           2.0   
0000d17a1447b25a01e42e1ac56b091bb7cbb06317be4cb...                           2.0   
00013181a0c5fc8f1ea38cd2b90fe8ad2fa8cad9d9f13e4...                           2.0   

                                                    B_10_ECDF Percentile Count_1  \
customer_ID                                                                        
00000fd6641609c6ece5454664794f0340ad84dddce9a26...                          10.0   
000041bdba6ecadd89a52d11886e8eaaec9325906c97233...                          10.0   
00007889e4fcd2614b6cbe7f8f3d2e5c728eca32d9eb8ad...                          10.0   
0000d17a1447b25a01e42e1ac56b091bb7cbb06317be4cb...                          10.0   
00013181a0c5fc8f1ea38cd2b90fe8ad2fa8cad9d9f13e4...                          10.0   

                                                    B_10_ECDF Percentile_0  \
customer_ID                                                                  
00000fd6641609c6ece5454664794f0340ad84dddce9a26...                0.295277   
000041bdba6ecadd89a52d11886e8eaaec9325906c97233...                0.196575   
00007889e4fcd2614b6cbe7f8f3d2e5c728eca32d9eb8ad...                0.046647   
0000d17a1447b25a01e42e1ac56b091bb7cbb06317be4cb...                0.114122   
00013181a0c5fc8f1ea38cd2b90fe8ad2fa8cad9d9f13e4...                0.008369   

                                                    B_10_ECDF Percentile_1  \
customer_ID                                                                  
00000fd6641609c6ece5454664794f0340ad84dddce9a26...                0.301472   
000041bdba6ecadd89a52d11886e8eaaec9325906c97233...                0.356053   
00007889e4fcd2614b6cbe7f8f3d2e5c728eca32d9eb8ad...                0.124556   
0000d17a1447b25a01e42e1ac56b091bb7cbb06317be4cb...                0.284792   
00013181a0c5fc8f1ea38cd2b90fe8ad2fa8cad9d9f13e4...                0.034970   

                                                    B_10_ECDF_0  B_10_ECDF_1  \
customer_ID                                                                    
00000fd6641609c6ece5454664794f0340ad84dddce9a26...     0.076923     0.153846   
000041bdba6ecadd89a52d11886e8eaaec9325906c97233...     0.076923     0.153846   
00007889e4fcd2614b6cbe7f8f3d2e5c728eca32d9eb8ad...     0.076923     0.153846   
0000d17a1447b25a01e42e1ac56b091bb7cbb06317be4cb...     0.076923     0.153846   
00013181a0c5fc8f1ea38cd2b90fe8ad2fa8cad9d9f13e4...     0.076923     0.153846   

                                                    B_10_ECDF_2  B_10_ECDF_3  \
customer_ID                                                                    
00000fd

In [26]:
features_df_joblib_opt.to_parquet('../../data/tsfel_features/test_13_months_boruta_selected_tsfel_stat_features.parquet', index=True, engine='pyarrow', compression='zstd')

In [22]:
features_df_joblib_opt = extract_tsfel_features_joblib_optimized(
        train_df[cols_to_append],
        customer_id_col='customer_ID',
        timestamp_col='end_of_month',
        features_to_extract=cfg_stat,
        n_jobs=13, # Use all cores
        verbose=10 # Set verbosity for progress updates and ETA
    )

# --- Display Results ---
print("\n--- Extracted Features (Optimized Joblib Method + Verbose Progress) ---")
if not features_df_joblib_opt.empty:
    print(f"Shape: {features_df_joblib_opt.shape}")
    try:
        display(features_df_joblib_opt.head())
    except NameError:
        print(features_df_joblib_opt.head())
else:
    print("Resulting DataFrame is empty.")

features_df_joblib_opt.to_parquet('../../data/tsfel_features/train_13_months_boruta_selected_tsfel_stat_features.parquet', index=True, engine='pyarrow', compression='zstd')

Input shape: (3486098, 49), Grouping by: customer_ID, Timestamp: end_of_month
Using provided TSFEL configuration dictionary.
Processing 289115 customer groups...
Setting up Joblib with 13 processes (verbose=10)...


[Parallel(n_jobs=13)]: Using backend LokyBackend with 13 concurrent workers.
[Parallel(n_jobs=13)]: Done   6 tasks      | elapsed:    3.9s
[Parallel(n_jobs=13)]: Done  15 tasks      | elapsed:    4.0s
[Parallel(n_jobs=13)]: Done  24 tasks      | elapsed:    4.1s
[Parallel(n_jobs=13)]: Done  35 tasks      | elapsed:    4.2s
[Parallel(n_jobs=13)]: Done  46 tasks      | elapsed:    4.2s
[Parallel(n_jobs=13)]: Batch computation too fast (0.19896662458010916s.) Setting batch_size=2.
[Parallel(n_jobs=13)]: Done  59 tasks      | elapsed:    4.3s
[Parallel(n_jobs=13)]: Done  72 tasks      | elapsed:    4.4s
[Parallel(n_jobs=13)]: Batch computation too fast (0.12594103813171387s.) Setting batch_size=4.
[Parallel(n_jobs=13)]: Done  96 tasks      | elapsed:    4.5s
[Parallel(n_jobs=13)]: Done 126 tasks      | elapsed:    4.7s
[Parallel(n_jobs=13)]: Done 190 tasks      | elapsed:    5.1s
[Parallel(n_jobs=13)]: Done 258 tasks      | elapsed:    5.5s
[Parallel(n_jobs=13)]: Done 334 tasks      | elap


Joblib parallel processing finished in 1764.70 seconds.
Successfully processed 253581 out of 289115 groups.
Preparing results for concatenation...
Concatenating reindexed results...
Concatenation complete.
Robust concatenation finished in 5056.34 seconds.
Final features DataFrame shape: (253581, 1457)

--- Extracted Features (Optimized Joblib Method + Verbose Progress) ---
Shape: (253581, 1457)


B_10_Absolute energy  \
customer_ID                                                                
0000099d6bd597052cdcda90ffabf56573fe9d7c79be5fb...              1.346612   
00001b22f846c82c51f6e3958ccd81970162bae8b007e80...              1.007482   
000084e5023181993c2e1b665ac88dbb1ce9ef621ec5370...              1.091303   
000098081fde4fd64bc4d503a5d6f86a0aedc425c96f523...              0.127795   
0000f99513770170a1aba690daeeb8a96da4a39f11fc27d...              0.002137   

                                                    B_10_Average power  \
customer_ID                                                              
0000099d6bd597052cdcda90ffabf56573fe9d7c79be5fb...           11.221769   
00001b22f846c82c51f6e3958ccd81970162bae8b007e80...            8.395682   
000084e5023181993c2e1b665ac88dbb1ce9ef621ec5370...            9.094188   
000098081fde4fd64bc4d503a5d6f86a0aedc425c96f523...            1.064961   
0000f99513770170a1aba690daeeb8a96da4a39f11fc27d...            0.017811   

                                                    B_10_ECDF Percentile Count_0  \
customer_ID                                                                        
0000099d6bd597052cdcda90ffabf56573fe9d7c79be5fb...                           2.0   
00001b22f846c82c51f6e3958ccd81970162bae8b007e80...                           2.0   
000084e5023181993c2e1b665ac88dbb1ce9ef621ec5370...                           2.0   
000098081fde4fd64bc4d503a5d6f86a0aedc425c96f523...                           2.0   
0000f99513770170a1aba690daeeb8a96da4a39f11fc27d...                           2.0   

                                                    B_10_ECDF Percentile Count_1  \
customer_ID                                                                        
0000099d6bd597052cdcda90ffabf56573fe9d7c79be5fb...                          10.0   
00001b22f846c82c51f6e3958ccd81970162bae8b007e80...                          10.0   
000084e5023181993c2e1b665ac88dbb1ce9ef621ec5370...                          10.0   
000098081fde4fd64bc4d503a5d6f86a0aedc425c96f523...                          10.0   
0000f99513770170a1aba690daeeb8a96da4a39f11fc27d...                          10.0   

                                                    B_10_ECDF Percentile_0  \
customer_ID                                                                  
0000099d6bd597052cdcda90ffabf56573fe9d7c79be5fb...                0.099804   
00001b22f846c82c51f6e3958ccd81970162bae8b007e80...                0.168061   
000084e5023181993c2e1b665ac88dbb1ce9ef621ec5370...                0.242979   
000098081fde4fd64bc4d503a5d6f86a0aedc425c96f523...                0.020843   
0000f99513770170a1aba690daeeb8a96da4a39f11fc27d...                0.005759   

                                                    B_10_ECDF Percentile_1  \
customer_ID                                                                  
0000099d6bd597052cdcda90ffabf56573fe9d7c79be5fb...                0.325464   
00001b22f846c82c51f6e3958ccd81970162bae8b007e80...                0.302099   
000084e5023181993c2e1b665ac88dbb1ce9ef621ec5370...                0.298666   
000098081fde4fd64bc4d503a5d6f86a0aedc425c96f523...                0.089057   
0000f99513770170a1aba690daeeb8a96da4a39f11fc27d...                0.014064   

                                                    B_10_ECDF_0  B_10_ECDF_1  \
customer_ID                                                                    
0000099d6bd597052cdcda90ffabf56573fe9d7c79be5fb...     0.076923     0.153846   
00001b22f846c82c51f6e3958ccd81970162bae8b007e80...     0.076923     0.153846   
000084e5023181993c2e1b665ac88dbb1ce9ef621ec5370...     0.076923     0.153846   
000098081fde4fd64bc4d503a5d6f86a0aedc425c96f523...     0.076923     0.153846   
0000f99513770170a1aba690daeeb8a96da4a39f11fc27d...     0.076923     0.153846   

                                                    B_10_ECDF_2  B_10_ECDF_3  \
customer_ID                                                                    
0000099

In [ ]:
# def extract_tsfel_features_parallel(df, customer_id_col, timestamp_col, features_to_extract=None, n_jobs=-1, verbose=10):
#     """
#     Extracts time-series features using TSFEL in parallel for each customer.

#     Args:
#         df (pd.DataFrame): Input DataFrame containing customer data and timestamps.
#                           It should be sorted by customer_id_col and timestamp_col
#                           for TSFEL to work correctly on sequential data.
#         customer_id_col (str): Name of the column containing customer IDs.
#         timestamp_col (str): Name of the column containing timestamps (like 'S_2').
#                                This column is used for sorting within groups but not
#                                directly by TSFEL unless specified in features_to_extract.
#                                TSFEL primarily works on the sequence of values in other columns.
#         features_to_extract (list or dict, optional): List of feature names or a TSFEL
#                                                      configuration dictionary.
#                                                      If None, uses TSFEL's default feature set.
#                                                      Defaults to None.
#         n_jobs (int, optional): Number of CPU cores to use for parallel processing.
#                                 -1 means using all available cores.
#                                 Defaults to -1.
#         verbose (int, optional): Verbosity level for joblib Parallel. Higher values
#                                  print more messages. Defaults to 10.

#     Returns:
#         pd.DataFrame: A DataFrame where each row corresponds to a customer_ID,
#                       and columns are the extracted TSFEL features.
#                       Returns None if the input DataFrame is empty or grouping fails.
#     """

#     if df.empty:
#         print("Input DataFrame is empty.")
#         return None

#     print(f"Input DataFrame shape: {df.shape}")
#     print(f"Grouping by: {customer_id_col}")
#     print(f"Sorting within groups by: {timestamp_col}")

#     # --- Preprocessing ---
#     # Ensure data is sorted correctly for time-series analysis within each group
#     # Create a copy to avoid modifying the original DataFrame outside the function if needed
#     df_sorted = df.sort_values(by=[customer_id_col, timestamp_col]).copy()

#     # Identify numeric columns for TSFEL (excluding ID and timestamp)
#     numeric_cols = df_sorted.select_dtypes(include=['number']).columns
#     cols_to_process_globally = [col for col in numeric_cols if col not in [customer_id_col, timestamp_col]] # Check potential target columns
#     if not cols_to_process_globally:
#         print("Error: No numeric columns (excluding ID/timestamp) found in the DataFrame to process.")
#         return None
#     print(f"Potential columns for feature extraction: {cols_to_process_globally}")


#     # --- Grouping ---
#     grouped_data = df_sorted.groupby(customer_id_col)
#     all_customer_ids = list(grouped_data.groups.keys())
#     print(f"Number of unique customers: {len(all_customer_ids)}")
#     if len(all_customer_ids) == 0:
#         print("No customers found after grouping.")
#         return None

#     # --- TSFEL Configuration ---
#     if features_to_extract is None:
#         # Using default configuration - extracts features from all numeric columns passed to it
#         cfg = tsfel.get_features_by_domain()
#         print("Using default TSFEL feature configuration.")
#     elif isinstance(features_to_extract, dict):
#         cfg = features_to_extract
#         print("Using provided TSFEL configuration dictionary.")
#         # If config specifies input_cols, ensure they exist
#         if "input_cols" in cfg and isinstance(cfg["input_cols"], list):
#              cols_to_process_globally = [col for col in cfg["input_cols"] if col in df_sorted.columns and col not in [customer_id_col, timestamp_col]]
#              if not cols_to_process_globally:
#                  print(f"Error: None of the specified 'input_cols' ({cfg['input_cols']}) are valid numeric columns in the DataFrame (excluding ID/timestamp).")
#                  return None
#              print(f"Using specified input columns for feature extraction: {cols_to_process_globally}")
#     else:
#          # Handle other potential formats or fall back to default
#          print("Warning: 'features_to_extract' format not a dict or None. Using default config.")
#          cfg = tsfel.get_features_by_domain()

#     # --- Parallel Processing Definition ---
#     def process_customer(customer_id, customer_data):
#         """Extracts TSFEL features for a single customer."""
#         try:
#             # Select only the columns relevant for this configuration
#             # For default config, TSFEL uses all numeric columns passed to it.
#             # For custom config with 'input_cols', use those specific columns.
#             if "input_cols" in cfg and isinstance(cfg["input_cols"], list):
#                  cols_to_use = [col for col in cfg["input_cols"] if col in customer_data.columns]
#             else:
#                  # Default: use all numeric columns available in this customer's data chunk
#                  numeric_cols_cust = customer_data.select_dtypes(include=['number']).columns
#                  cols_to_use = [col for col in numeric_cols_cust if col not in [customer_id_col, timestamp_col]]


#             if not cols_to_use:
#                  # This might happen if a customer only has non-numeric data or only ID/timestamp
#                  print(f"Info: No numeric columns to process for customer {customer_id}. Skipping.")
#                  return None

#             # Ensure there's enough data points if TSFEL requires a minimum window size
#             # (Some features might need > 1 data point)
#             if len(customer_data) < 2: # Example threshold, adjust if needed based on features
#                  print(f"Info: Insufficient data points ({len(customer_data)}) for customer {customer_id}. Skipping.")
#                  return None

#             # Extract features.
#             features_df = tsfel.time_series_features_extractor(cfg, customer_data[cols_to_use], verbose=0) # verbose=0 inside the loop

#             # Check if feature extraction returned an empty DataFrame (can happen)
#             if features_df.empty:
#                  print(f"Info: Feature extraction returned empty result for customer {customer_id}. Skipping.")
#                  return None

#             # Add the customer ID to the results
#             features_df[customer_id_col] = customer_id
#             # Set customer ID as index temporarily for easier concatenation later
#             features_df = features_df.set_index(customer_id_col)
#             return features_df
#         except Exception as e:
#             # Capture more specific errors if possible
#             print(f"Error processing customer {customer_id}: {type(e).__name__} - {e}")
#             # You might want to log the full traceback for debugging
#             import traceback
#             print(traceback.format_exc())
#             return None

#     # --- Parallel Execution ---
#     if n_jobs == -1:
#         num_cores = multiprocessing.cpu_count()
#         print(f"Using {num_cores} cores for parallel processing.")
#     else:
#         num_cores = n_jobs
#         print(f"Using {n_jobs} core(s) for parallel processing.")

#     print("Starting parallel feature extraction...")
#     start_time = time.time()

#     results = Parallel(n_jobs=num_cores, verbose=verbose)(
#         delayed(process_customer)(customer_id, group_df)
#         for customer_id, group_df in grouped_data
#     )

#     end_time = time.time()
#     print(f"Parallel processing finished in {end_time - start_time:.2f} seconds.")

#     # --- Combine Results ---
#     successful_results = [res for res in results if res is not None and not res.empty]

#     if not successful_results:
#         print("No features were successfully extracted for any customer.")
#         return None

#     print(f"Successfully processed {len(successful_results)} out of {len(all_customer_ids)} customers.")
#     print("Concatenating results...")
#     try:
#         final_features_df = pd.concat(successful_results)
#         print("Concatenation complete.")
#     except Exception as e:
#         print(f"Error during final concatenation: {e}")
#         # Fallback or further debugging needed here if concat fails
#         return None

#     # Optional: Reset index to have customer_id as a column
#     # final_features_df = final_features_df.reset_index()

#     print(f"Final features DataFrame shape: {final_features_df.shape}")

#     return final_features_df

In [ ]:
print("--- Running Extraction with Default Config ---")

# Using %%time magic command to time the cell execution
# Note: n_jobs=2 is used here for stability in diverse environments;
# Use n_jobs=-1 for maximum speed on your machine with larger data.
%time features_df_default = extract_tsfel_features_parallel(val_df, customer_id_col='customer_ID', timestamp_col='end_of_month', n_jobs=-1, verbose=5, features_to_extract=cfg_stat_temp)

if features_df_default is not None:
    print("\n--- Extracted Features (Default Config) ---")
    print(f"Shape: {features_df_default.shape}")
    print("First 5 rows:")
    display(features_df_default.head())
    # print("\nLast 5 rows:")
    # display(features_df_default.tail()) # Useful to check different customers
else:
    print("\nFeature extraction with default config failed or produced no results.")

--- Running Extraction with Default Config ---
Input DataFrame shape: (387137, 89)
Grouping by: customer_ID
Sorting within groups by: end_of_month
Potential columns for feature extraction: ['D_39', 'B_1', 'R_1', 'B_4', 'B_5', 'R_2', 'D_47', 'B_7', 'D_51', 'B_9', 'R_3', 'B_10', 'S_5', 'B_11', 'S_6', 'R_4', 'B_12', 'S_8', 'R_5', 'D_58', 'B_14', 'D_60', 'S_11', 'D_65', 'B_18', 'S_12', 'R_6', 'S_13', 'B_21', 'D_71', 'S_15', 'B_23', 'P_4', 'D_75', 'B_24', 'R_7', 'R_8', 'S_16', 'R_10', 'R_11', 'S_17', 'R_12', 'B_28', 'R_13', 'R_14', 'R_15', 'R_16', 'S_18', 'D_86', 'R_17', 'R_18', 'B_31', 'S_19', 'R_19', 'B_32', 'S_20', 'R_20', 'R_21', 'R_22', 'R_23', 'D_92', 'D_93', 'D_94', 'R_24', 'R_25', 'D_96', 'S_26', 'D_102', 'B_36', 'B_37', 'B_40', 'D_127', 'B_41', 'D_133', 'R_28', 'D_140', 'D_144', 'D_126_-1.0', 'D_126_0.0', 'D_126_1.0', 'D_126_nan', 'D_63_CL', 'D_63_CO', 'D_63_CR', 'D_63_XL', 'D_63_XM', 'D_63_XZ']
Number of unique customers: 32124
Using provided TSFEL configuration dictionary.
Using 

[Parallel(n_jobs=20)]: Using backend LokyBackend with 20 concurrent workers.
[Parallel(n_jobs=20)]: Done  32 tasks      | elapsed:    6.3s
[Parallel(n_jobs=20)]: Done 122 tasks      | elapsed:    7.3s
[Parallel(n_jobs=20)]: Done 248 tasks      | elapsed:    8.7s
[Parallel(n_jobs=20)]: Done 410 tasks      | elapsed:   10.5s
[Parallel(n_jobs=20)]: Done 608 tasks      | elapsed:   12.8s
[Parallel(n_jobs=20)]: Done 842 tasks      | elapsed:   15.4s
[Parallel(n_jobs=20)]: Done 1112 tasks      | elapsed:   18.3s
[Parallel(n_jobs=20)]: Done 1418 tasks      | elapsed:   21.6s
[Parallel(n_jobs=20)]: Done 1760 tasks      | elapsed:   25.5s
[Parallel(n_jobs=20)]: Done 2138 tasks      | elapsed:   29.7s
[Parallel(n_jobs=20)]: Done 2552 tasks      | elapsed:   34.3s
[Parallel(n_jobs=20)]: Done 3002 tasks      | elapsed:   39.5s
[Parallel(n_jobs=20)]: Done 3488 tasks      | elapsed:   45.1s
[Parallel(n_jobs=20)]: Done 4010 tasks      | elapsed:   51.3s
[Parallel(n_jobs=20)]: Done 4568 tasks      | e

Parallel processing finished in 397.64 seconds.
Successfully processed 28274 out of 32124 customers.
Concatenating results...


In [ ]:
features_df_default

In [ ]:
2+2

4